In [65]:
def get_node_moves(solution, flag):
    solution = solution[:-1]
    n = len(solution)
    moves = list()
    if flag == 'intra':
        for i in range(len(solution) - 1):
            for j in range(i+1, len(solution)):
                moves.append((solution[i], solution[j]))
    elif flag == 'inter':
        outer_nodes = list(set(range(int(n*2))) - set(solution))
        for i in range(len(solution)):
            for j in range(len(outer_nodes)):
                moves.append((solution[i], outer_nodes[j]))   
    return moves

In [67]:
def get_edge_moves(solution):
    solution = solution[:-1]
    moves = list()
    for i in range(len(solution) - 1):
        for j in range(i + 2, len(solution) - 1):
            edge1 = (solution[i], solution[i + 1])
            edge2 = (solution[j], solution[j + 1])
            moves.append((edge1, edge2))
    return moves

In [89]:
def compute_node_swap_delta(solution, swap, distance_matrix, costs, flag):
    new_solution = solution[:-1]
    n = len(new_solution)
    if flag == 'intra':
        node1, node2 = swap
        node1_idx, node2_idx = new_solution.index(node1), new_solution.index(node2)
        prev_node1, next_node1 = new_solution[node1_idx-1], new_solution[(node1_idx+1)%n]
        prev_node2, next_node2 = new_solution[node2_idx-1], new_solution[(node2_idx+1)%n]

        old_cost = distance_matrix[prev_node1][node1] + distance_matrix[node1][next_node1] + \
                   distance_matrix[prev_node2][node2] + distance_matrix[node2][next_node2]
        
        new_cost = distance_matrix[prev_node1][node2] + distance_matrix[node2][next_node1] + \
                   distance_matrix[prev_node2][node1] + distance_matrix[node1][next_node2]
        
        delta = old_cost - new_cost
        new_solution[node1_idx], new_solution[node2_idx] = new_solution[node2_idx], new_solution[node1_idx]
        new_solution.append(new_solution[0])
    elif flag == 'inter':
        solution_node, outer_node = swap
        solution_node_idx = new_solution.index(solution_node)
        prev_solution_node, next_solution_node = new_solution[solution_node_idx-1], new_solution[(solution_node_idx+1)%n]

        old_cost = distance_matrix[prev_solution_node][solution_node] + distance_matrix[solution_node][next_solution_node] + \
                   costs[solution_node]
        new_cost = distance_matrix[prev_solution_node][outer_node] + distance_matrix[outer_node][next_solution_node] + \
                   costs[outer_node]
        
        delta = old_cost - new_cost
        new_solution[solution_node_idx] = outer_node
        new_solution.append(new_solution[0])

    return delta, new_solution

In [69]:
def compute_edge_swap_delta(solution, swap, distance_matrix):
    (edge1_start, edge1_end), (edge2_start, edge2_end) = swap
    idx_edge1_start = solution.index(edge1_start)
    idx_edge2_start = solution.index(edge2_start)
    
    old_distance = distance_matrix[edge1_start][edge1_end] + distance_matrix[edge2_start][edge2_end]
    new_distance = distance_matrix[edge1_start][edge2_start] + distance_matrix[edge1_end][edge2_end]
    
    delta = old_distance - new_distance


    if idx_edge1_start < idx_edge2_start:
        new_solution = solution[:idx_edge1_start+1] + solution[idx_edge1_start+1:idx_edge2_start+1][::-1] + \
                       solution[idx_edge2_start+1:] 
    else:
        new_solution = solution[:idx_edge2_start+1] + solution[idx_edge2_start+1:idx_edge1_start+1][::-1] + \
                       solution[idx_edge1_start+1:]
    return delta, new_solution